In [1]:
from pandas import DataFrame
from data201 import db_connection, df_query

In [2]:
conn = db_connection('sheql_wh.ini')
cursor = conn.cursor()

In [3]:
def display_results(cursor):
    """
    If there are results in the cursor from a call to a
    stored procedure, display the results in a dataframe. 
    """
    columns = result.description
    
    if columns == None:
        return 
    
    else:
        column_names = [column_info[0] 
                        for column_info in columns]

        rows = cursor.fetchall()
    
        if len(rows) > 0:
            df = DataFrame(rows)
            df.columns = column_names
            
            display(df)
        
        else:
            return

# Create a query to get all schools 

In [4]:
cursor.execute('DROP PROCEDURE IF EXISTS get_all_schools')

cursor.execute(
    """
    CREATE PROCEDURE get_all_schools()
    BEGIN
        SELECT school_id, name
        FROM school_dim
        ORDER BY name;
    END
    """
)

In [5]:
cursor.callproc('get_all_schools')

for result in cursor.stored_results():
    display_results(result)

,school_id,name
0,5,Campbell Middle School
1,3,Coffey Elementary School
2,4,Griffin Elementary School
3,7,Johnson Middle School
4,9,Larsen High School
5,8,Lewis High School
6,10,Mclaughlin Special Education School
7,1,Melton Elementary School
8,6,Powell Middle School
9,2,Singh Elementary School


# Query to load appropriate grade levels from a certain school

In [6]:
cursor.execute('DROP PROCEDURE IF EXISTS get_grade_levels')

cursor.execute(
    """
    CREATE PROCEDURE get_grade_levels(
        IN schoolId INT
        )
    BEGIN
        SELECT DISTINCT sd.grade_level
        FROM student_dim sd
        JOIN student_performance_fact spf ON spf.student_id = sd.student_id
        WHERE spf.school_id = schoolId
        ORDER BY sd.grade_level;
    END
    """
)

In [7]:
schoolId = 1
cursor.callproc('get_grade_levels', (schoolId,))

for result in cursor.stored_results():
    display_results(result)

,grade_level
0,1
1,2
2,3
3,4
4,5
5,K


# Query to compute the average score for a specific school, grade level, and assignment 

In [8]:
cursor.execute('DROP PROCEDURE IF EXISTS get_avg_score')

cursor.execute(
    """
    CREATE PROCEDURE get_avg_score(
        IN schoolId INT,
        IN gradeLevel VARCHAR(4),
        IN gradeType VARCHAR(20)
        )
    BEGIN
        SELECT ROUND(AVG(spf.score), 2) AS avg_score
        FROM student_performance_fact spf
        JOIN student_dim sd ON spf.student_id = sd.student_id
        WHERE spf.school_id = schoolId
        AND sd.grade_level = gradeLevel
        AND LOWER(spf.grade_type) = gradeType;
    END
    """
)

In [9]:
schoolId = 1
gradeLevel = '4'
gradeType = 'mid exam'

cursor.callproc('get_avg_score', (schoolId, gradeLevel, gradeType))

for result in cursor.stored_results():
    display_results(result)

,avg_score
0,79.19


# Query to Display attendance rate at a specific school, for a certain grade, on a specific day 

In [10]:
cursor.execute('DROP PROCEDURE IF EXISTS get_attendance_rate')

cursor.execute(
    """
    CREATE PROCEDURE get_attendance_rate(
        IN schoolId INT,
        IN gradeLevel VARCHAR(4),
        IN date VARCHAR(20)
        )
    BEGIN
        SELECT
        ROUND(100 * SUM(CASE WHEN saf.status = 'present' THEN 1 ELSE 0 END) / COUNT(*), 1) AS attendance_rate
        FROM student_attendance_fact saf
        JOIN student_dim sd ON saf.student_id = sd.student_id
        WHERE saf.school_id = schoolId
        AND sd.grade_level = gradeLevel
        AND saf.date_id = TO_DAYS(STR_TO_DATE(date, '%Y-%m-%d'));
    END
    """
)

In [11]:
schoolId = 10
gradeLevel = '9'
date = '2025-05-16'

cursor.callproc('get_attendance_rate', (schoolId, gradeLevel, date))

for result in cursor.stored_results():
    display_results(result)

,attendance_rate
0,84.6


In [12]:
cursor.close()
conn.close()